In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [7]:
df.sample(12)

,age,gender,fever,cough,city,has_covid
79,48,Female,103.0,Mild,Kolkata,Yes
14,51,Male,104.0,Mild,Bangalore,No
43,22,Female,99.0,Mild,Bangalore,Yes
20,12,Male,98.0,Strong,Bangalore,No
84,69,Female,98.0,Strong,Mumbai,No
10,75,Female,NaN,Mild,Delhi,No
85,16,Female,103.0,Mild,Bangalore,Yes
9,64,Female,101.0,Mild,Delhi,No
66,51,Male,104.0,Mild,Kolkata,No
37,55,Male,100.0,Mild,Kolkata,No


In [5]:
df.shape

(100, 6)

In [11]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [12]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [14]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [15]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [18]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [19]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [20]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

In [21]:
X_train_transformed

array([[ 25.        , 104.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [  5.        , 100.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 84.        , 101.04109589,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 73.        , 103.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 64.        ,  98.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 20.        , 101.04109589,   0.        ,   0.        ,
          0.        ,   1.        ,   1.        ],
       [ 59.        ,  99.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   1.        ],
       [ 71.        ,  98.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   1.        ],
       [ 11.        , 100.        ,   1.        ,   0.        ,
          0.    

In [32]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [33]:
transformer.fit_transform(X_train).shape

(80, 7)